# Stacking (ensemble) machine learning models to improve forecasting

In machine learning, stacking is an ensemble learning technique that combines multiple models to reduce their biases and improve predictive performance. More specifically, the predictions of each model (base models) are stacked and used as input to a final model (meta model) to compute the prediction.

Stacking is effective because it leverages the strengths of different algorithms and attempts to mitigate their individual weaknesses. By combining several models, it can capture complex patterns in the data and improve prediction accuracy.

However, stacking can be computationally expensive and requires careful tuning to avoid overfitting. To this end, it is highly recommended to train the final estimator through cross-validation. In addition, obtaining diverse and well-performing base models is critical to the success of the stacking technique.

<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4; padding-left: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00b8d4;"></i>
    <b style="color: #00b8d4;">&#9998 Note</b>
</p>

See <a href="https://skforecast.org/latest/faq/cyclical-features-time-series.html" target="_blank">Stacking (ensemble) machine learning models to improve forecasting</a> for a more detailed example of stacking models.

</div>

## StackingRegressor

With scikit-learn it is very easy to combine multiple regressors thanks to its [StackingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html#sklearn.ensemble.StackingRegressor) class.

The `estimators` parameter corresponds to the list of the estimators (base learners) which are stacked together in parallel on the input data. It should be given as a list of names and estimators. The `final_estimator` (meta model) will use the predictions of the estimators as input.

In [4]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.templates.default = "seaborn"
plt.style.use('seaborn-v0_8-darkgrid')

# Modelling and Forecasting
# ==============================================================================
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble  import StackingRegressor
from sklearn.model_selection  import KFold

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.datasets import fetch_dataset

# Configuration warnings
# ==============================================================================
import warnings

In [ ]:
# Data
# ==============================================================================

In [6]:
#  Create stacking regressor
# ==============================================================================
params_ridge = {'alpha': 0.001}
params_lgbm = {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500}

estimators = [
    ('ridge', Ridge(**params_ridge)),
    ('lgbm', LGBMRegressor(random_state=42, **params_lgbm)),
]
stacking_regressor = StackingRegressor(
    estimators = estimators,
    final_estimator = Ridge(),
    cv = KFold(n_splits=5, shuffle=False)

)
stacking_regressor

StackingRegressor(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                  estimators=[('ridge', Ridge(alpha=0.001)),
                              ('lgbm',
                               LGBMRegressor(max_depth=10, n_estimators=500,
                                             random_state=42))],
                  final_estimator=Ridge())

In [7]:
# Create forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = stacking_regressor,
                 lags = 72
             )

In [ ]:
# Backtesting on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                  = data['users'],
                            initial_train_size = len(data.loc[:end_validation]),
                            fixed_train_size   = False,
                            steps              = 36,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            n_jobs             = 'auto',
                            verbose            = False
                      )        

print(f"Backtest error: {metric:.2f}")

## Hiperparameters search of StackingRegressor

When using `StackingRegressor`, the hyperparameters of individual regressors must be prefixed with the name of the regressor followed by two underlines. For example, the hyperparameter `alpha`` of the Ridge regressor must be specified as `ridge__alpha``. The hyperparameter of the final estimator must be specified with the `final_estimator__` prefix.

In [ ]:
# Grid search of hyperparameters and lags
# ==============================================================================
param_grid = {
    'ridge__alpha': [0.1, 1, 10],
    'lgbm__n_estimators': [100, 500],
    'lgbm__max_depth': [3, 5, 10],
    'lgbm__learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [24, 48, 72]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = data.loc[:end_validation, 'users'],
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 36,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(data.loc[:end_train]),
                   fixed_train_size   = False,
                   return_best        = True,
                   n_jobs             = 'auto',
                   verbose            = False
               )

results_grid.head()